In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, TensorDataset

In [37]:
train_dataset = MNIST(root='./data', train=True, download=True)
test_dataset = MNIST(root='./data', train=False, download=True)
# Для обучающего набора преобразуем массив в np вид и нормализуем, чтобы оттенок пикселя был дробным числом от 0 до 1
train_images_tensor = train_dataset.data.clone().detach() / 255
test_images_tensor = test_dataset.data.clone().detach() / 255
train_labels_tensor = train_dataset.targets.clone().detach()
test_labels_tensor = test_dataset.targets.clone().detach()

train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

In [148]:
num_images = len(train_labels_tensor)
print("Количество изображений в train_dataset:", num_images)

Количество изображений в train_dataset: 60000


In [166]:
# динамическое количество слоев, по умолчанию - 1 скрытый слой
class DynamicNN(nn.Module):
    def __init__(self, layers_count = 3, activation_func = torch.relu):
        super(DynamicNN, self).__init__()
        if layers_count <= 0:
            raise Exception("layers_count must be greater than 0")
        self.activation_func, self.layers_count = activation_func, layers_count
        self.flatten = nn.Flatten()
        self.accuracy = 0
        in_size = 28 * 28
        self.fc_layers = []
        layer_sizes = []
        while layers_count > 0:
            layers_count = layers_count - 1
            out_size = 128 * (2 ** layers_count)
            self.fc_layers.append(nn.Linear(in_size, out_size))
            layer_sizes.append([in_size, out_size])
            in_size = out_size
        self.fc_last = nn.Linear(out_size, 10)  # Выходной слой с 10 нейронами
        print('Создана модель с функцией активации %s, Слои: %s' % (activation_func.__name__, layer_sizes + [[out_size, 10]]))

    def forward(self, x):
        x = self.flatten(x)
        for layer in self.fc_layers:
            x = self.activation_func(layer(x))
        return self.fc_last(x)

    def __repr__(self):
        return f"Количество слоев {self.layers_count}, Функция активации: '{self.activation_func.__name__}'"

In [168]:
models = {}
activation_functions = [torch.relu, torch.sigmoid, torch.tanh]
for activation_function in activation_functions:
    model_list = []
    for i in range(1, 4):
        model = DynamicNN(layers_count = i, activation_func = activation_function)
        model_list.append(model)
    models[activation_function.__name__] = model_list

Создана модель с функцией активации relu, Слои: [[784, 128], [128, 10]]
Создана модель с функцией активации relu, Слои: [[784, 256], [256, 128], [128, 10]]
Создана модель с функцией активации relu, Слои: [[784, 512], [512, 256], [256, 128], [128, 10]]
Создана модель с функцией активации sigmoid, Слои: [[784, 128], [128, 10]]
Создана модель с функцией активации sigmoid, Слои: [[784, 256], [256, 128], [128, 10]]
Создана модель с функцией активации sigmoid, Слои: [[784, 512], [512, 256], [256, 128], [128, 10]]
Создана модель с функцией активации tanh, Слои: [[784, 128], [128, 10]]
Создана модель с функцией активации tanh, Слои: [[784, 256], [256, 128], [128, 10]]
Создана модель с функцией активации tanh, Слои: [[784, 512], [512, 256], [256, 128], [128, 10]]


In [182]:
def learn(model):
    criterion, num_epochs, epoch_losses = nn.CrossEntropyLoss(), 30, []
    optimizer = optim.SGD(model.parameters(), lr = 1)
    for epoch in range(num_epochs):
        model.train()  # Установка модели в режим обучения
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()  # Обнуление градиентов
            outputs = model(images)  # Получение выхода модели
            loss = criterion(outputs, labels)  # Вычисление потерь
            loss.backward()  # Обратное распространение ошибки
            optimizer.step()  # Обновление весов
            running_loss += loss.item() * images.size(0)  # Учитываем потери по всем изображениям
        epoch_losses.append(running_loss / len(train_loader.dataset))  # Вычисляем среднюю потерю за эпоху
    print(f"Потери по эпохам:", epoch_losses)

In [183]:
i = 0
for activation_function_name, model_list in models.items():
    for model in model_list:
        i = i + 1
        print(f"Обучаем модель №{i}: {model}")
        learn(model)

Обучаем модель №1: Количество слоев 1, Функция активации: 'relu'
Потери по эпохам: [0.4350205369949341, 0.43196364760398864, 0.4290453776359558, 0.42647132250467934, 0.42370104482968646, 0.42167667597134906, 0.41990425231456757, 0.41811958292325335, 0.4163909602165222, 0.41475950441360476, 0.4134699706395467, 0.4121733752091726, 0.4108813715616862, 0.4098699740568797, 0.4085444192647934, 0.4078015597025553, 0.4067841175556183, 0.4058718619187673, 0.40502447135448455, 0.40423122448921206, 0.4036321506023407, 0.4028745835940043, 0.4023299532413483, 0.40157244108517964, 0.40119812258084614, 0.40044063800175983, 0.3999549703121185, 0.39925758010546364, 0.3990261175950368, 0.3982356737057368]
Обучаем модель №2: Количество слоев 2, Функция активации: 'relu'
Потери по эпохам: [0.6136209622383118, 0.6052430678685506, 0.597338964621226, 0.5904596961975098, 0.5841609942436218, 0.5787699253082276, 0.5736467922210693, 0.5687632571538289, 0.5643510363260905, 0.5601607686042785, 0.5563319842020671, 

In [184]:
def assessment(model):
    model.eval()
    total = len(test_labels_tensor)
    with torch.no_grad():
        outputs = model(test_images_tensor)
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == test_labels_tensor).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [185]:
i = 0
for activation_function_name, model_list in models.items():
    for model in model_list:
        i = i + 1
        print(f"Оцениваем модель №{i}: {model}")
        accuracy = assessment(model)
        model.accuracy = accuracy
        print(f'Точность при проверке на тестовом наборе: {model.accuracy:.2f}%')

Оцениваем модель №1: Количество слоев 1, Функция активации: 'relu'
Точность при проверке на тестовом наборе: 87.99%
Оцениваем модель №2: Количество слоев 2, Функция активации: 'relu'
Точность при проверке на тестовом наборе: 85.19%
Оцениваем модель №3: Количество слоев 3, Функция активации: 'relu'
Точность при проверке на тестовом наборе: 78.89%
Оцениваем модель №4: Количество слоев 1, Функция активации: 'sigmoid'
Точность при проверке на тестовом наборе: 87.39%
Оцениваем модель №5: Количество слоев 2, Функция активации: 'sigmoid'
Точность при проверке на тестовом наборе: 44.75%
Оцениваем модель №6: Количество слоев 3, Функция активации: 'sigmoid'
Точность при проверке на тестовом наборе: 9.82%
Оцениваем модель №7: Количество слоев 1, Функция активации: 'tanh'
Точность при проверке на тестовом наборе: 90.35%
Оцениваем модель №8: Количество слоев 2, Функция активации: 'tanh'
Точность при проверке на тестовом наборе: 89.93%
Оцениваем модель №9: Количество слоев 3, Функция активации: 'tan